In [1]:
import sys
import os

import numpy as np
import pandas as pd


pd.set_option("display.max_columns", None)

# Agregar la carpeta src al path para importar el script de procesamiento de datos
sys.path.append(os.path.join(os.getcwd(), '../src'))


data = pd.read_csv('../data/houses_Madrid.csv', sep=',', header=0, index_col=0)
houses = pd.read_csv('../data/madrid_houses_clean.csv', sep=',', header=0, index_col=0)

data.head()

,id,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,latitude,longitude,raw_address,is_exact_address_hidden,street_name,street_number,portal,floor,is_floor_under,door,neighborhood_id,operation,rent_price,rent_price_by_area,is_rent_price_known,buy_price,buy_price_by_area,is_buy_price_known,house_type_id,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,are_pets_allowed,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_furnished,is_kitchen_equipped,is_accessible,has_green_zones,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
0,21742,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64.0,60.0,2,1.0,NaN,NaN,NaN,NaN,"Calle de Godella, 64",False,Calle de Godella,64,NaN,3,False,NaN,Neighborhood 135: San Cristóbal (1308.89 €/m2)...,sale,471,NaN,False,85000,1328,True,HouseType 1: Pisos,False,False,1960.0,NaN,NaN,NaN,True,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,False,NaN,NaN,NaN,NaN,False,True,False,False
1,21741,Piso en venta en calle de la del Manojo de Rosas,"Los Ángeles, Madrid",70.0,NaN,3,1.0,NaN,NaN,NaN,NaN,Calle de la del Manojo de Rosas,True,Calle de la del Manojo de Rosas,NaN,NaN,4,False,NaN,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,sale,666,NaN,False,129900,1856,True,HouseType 1: Pisos,True,False,NaN,NaN,NaN,NaN,NaN,True,True,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,en trámite,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21740,"Piso en venta en calle del Talco, 68","San Andrés, Madrid",94.0,54.0,2,2.0,NaN,NaN,NaN,NaN,"Calle del Talco, 68",False,Calle del Talco,68,NaN,1,False,NaN,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,sale,722,NaN,False,144247,1535,True,HouseType 1: Pisos,False,False,NaN,False,True,NaN,NaN,True,True,True,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,no indicado,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21739,Piso en venta en calle Pedro Jiménez,"San Andrés, Madrid",64.0,NaN,2,1.0,NaN,NaN,NaN,NaN,Calle Pedro Jiménez,True,Calle Pedro Jiménez,NaN,NaN,Bajo,True,NaN,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,sale,583,NaN,False,109900,1717,True,HouseType 1: Pisos,False,False,1955.0,NaN,NaN,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,True,NaN,NaN,True,NaN,en trámite,False,NaN,NaN,NaN,NaN,False,False,True,False
4,21738,Piso en venta en carretera de Villaverde a Val...,"Los Rosales, Madrid",108.0,90.0,2,2.0,NaN,NaN,NaN,NaN,Carretera de Villaverde a Vallecas,True,Carretera de Villaverde a Vallecas,NaN,NaN,4,False,NaN,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,sale,1094,NaN,False,260000,2407,True,HouseType 1: Pisos,False,False,2003.0,NaN,NaN,NaN,True,True,True,True,NaN,True,NaN,NaN,True,NaN,NaN,NaN,True,en trámite,True,NaN,NaN,True,0.0,True,True,True,True


In [2]:
data_map = data.loc[:, ['id','latitude', 'longitude', 'raw_address', 'subtitle']]


In [3]:
data_map.shape

(21742, 5)

Tendremos que enviar cada dirección a un servidor de geolocalización para obtener la latitud y la longitud (y la altitud, pero no la necesitaremos aquí). Hacer esto para múltiples direcciones llevará tiempo, así que tenemos que asegurarnos de que no estamos enviando información irrelevante.

Empecemos por eliminar los valores nulos y las direcciones duplicadas.

In [4]:
(data_map.raw_address.isnull()).sum()

np.int64(5465)

In [5]:
data_map.dropna(subset=['raw_address'],inplace=True)

In [6]:
data_map.raw_address.duplicated().sum()

np.int64(6611)

In [7]:
data_map.drop_duplicates(subset=['raw_address'], inplace=True)
data_map.shape

(9666, 5)

Sólo nos quedan menos de diez mil filas.

Crearemos una nueva columna con el nombre de la calle y su comarca.

In [8]:
data_map['address'] = data_map.raw_address + ', ' + data_map.subtitle
data_map.drop(columns=['raw_address', 'subtitle'], inplace=True)
data_map.reset_index(drop=True, inplace=True)

In [9]:
data_map.head(10)

,id,latitude,longitude,address
0,21742,NaN,NaN,"Calle de Godella, 64, San Cristóbal, Madrid"
1,21741,NaN,NaN,"Calle de la del Manojo de Rosas, Los Ángeles, ..."
2,21740,NaN,NaN,"Calle del Talco, 68, San Andrés, Madrid"
3,21739,NaN,NaN,"Calle Pedro Jiménez, San Andrés, Madrid"
4,21738,NaN,NaN,"Carretera de Villaverde a Vallecas, Los Rosale..."
5,21737,NaN,NaN,"geologia, San Andrés, Madrid"
6,21736,NaN,NaN,"Avenida Real de Pinto, San Andrés, Madrid"
7,21733,NaN,NaN,"Calle de Martinez Oviol, Los Rosales, Madrid"
8,21732,NaN,NaN,"De la Plata - Villaverde, San Andrés, Madrid"
9,21730,NaN,NaN,"Calle de la Unanimidad, 67, Los Rosales, Madrid"


Usaremos la librería 'geopy' para acceder a un servicio de geocodificación. Estos servicios proporcionan APIs y geopy es una librería que proporciona implementaciones para ellos.

Usaremos Nominatim que trabaja con datos de OpenStreetMap para encontrar localizaciones por su nombre. También puede hacer el proceso inverso, encontrar una dirección a partir de sus coordenadas.

Tenemos que especificar un nombre en el campo 'user-agent', por ejemplo, nuestro correo electrónico.

Traducción realizada con la versión gratuita del traductor DeepL.com

In [10]:
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent="pepeflores935@gmail.com")

location1 = locator.geocode("Calle de Godella, 64, San Cristóbal, Madrid")
location2 = locator.reverse("43.3624, -5.8444")

In [11]:
dir(location1)[-6:]

['address', 'altitude', 'latitude', 'longitude', 'point', 'raw']

In [12]:
print(location1.address)
print(location1.altitude)
print(location1.latitude)
print(location1.longitude)
print(location1.point)
print(location1.raw)

Calle de Godella, San Cristóbal, Villaverde, Madrid, Comunidad de Madrid, 28021, España
0.0
40.34069258072084
-3.6867978496595586
40 20m 26.4933s N, 3 41m 12.4723s W
{'place_id': 272779423, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 122397267, 'lat': '40.34069258072084', 'lon': '-3.6867978496595586', 'class': 'highway', 'type': 'residential', 'place_rank': 26, 'importance': 0.053419141680018274, 'addresstype': 'road', 'name': 'Calle de Godella', 'display_name': 'Calle de Godella, San Cristóbal, Villaverde, Madrid, Comunidad de Madrid, 28021, España', 'boundingbox': ['40.3406836', '40.3407016', '-3.6873026', '-3.6862931']}


In [13]:
print(location2.address)
print(location2.altitude)
print(location2.latitude)
print(location2.longitude)
print(location2.point)
print(location2.raw)

2, Plaza de Alfonso II El Casto (Plaza de la Catedral), Colonia Marqués de San Feliz, Casco Antiguo, Centro y casco histórico, Oviedo, Asturias / Asturies, 33080, España
0.0
43.362025900000006
-5.844274081250001
43 21m 43.2932s N, 5 50m 39.3867s W
{'place_id': 269960730, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 333079965, 'lat': '43.362025900000006', 'lon': '-5.844274081250001', 'class': 'building', 'type': 'apartments', 'place_rank': 30, 'importance': 6.877622644383944e-05, 'addresstype': 'building', 'name': '', 'display_name': '2, Plaza de Alfonso II El Casto (Plaza de la Catedral), Colonia Marqués de San Feliz, Casco Antiguo, Centro y casco histórico, Oviedo, Asturias / Asturies, 33080, España', 'address': {'house_number': '2', 'road': 'Plaza de Alfonso II El Casto (Plaza de la Catedral)', 'neighbourhood': 'Colonia Marqués de San Feliz', 'suburb': 'Casco Antiguo', 'city_district': 'Centro y casco histórico', 'ci

In [14]:
from geopy.extra.rate_limiter import RateLimiter

#function to delay calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
#save all the data in a new columns
data_map['location'] = data_map.loc[0:15]['address'].apply(geocode)

In [15]:
data_small = data_map.loc[0:15]
data_small['location'][0]

Location(Calle de Godella, San Cristóbal, Villaverde, Madrid, Comunidad de Madrid, 28021, España, (40.34069258072084, -3.6867978496595586, 0.0))

In [16]:
data_small['latitude'] = data_small['location'].apply(lambda loc: loc.latitude if loc else None)
data_small['longitude'] = data_small['location'].apply(lambda loc: loc.longitude if loc else None)

C:\Users\HP\AppData\Local\Temp\ipykernel_12988\1122140813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_small['latitude'] = data_small['location'].apply(lambda loc: loc.latitude if loc else None)
C:\Users\HP\AppData\Local\Temp\ipykernel_12988\1122140813.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_small['longitude'] = data_small['location'].apply(lambda loc: loc.longitude if loc else None)


In [17]:
data_small

,id,latitude,longitude,address,location
0,21742,40.340693,-3.686798,"Calle de Godella, 64, San Cristóbal, Madrid","(Calle de Godella, San Cristóbal, Villaverde, ..."
1,21741,40.355341,-3.697690,"Calle de la del Manojo de Rosas, Los Ángeles, ...","(Calle de La del Manojo de Rosas, Los Ángeles,..."
2,21740,NaN,NaN,"Calle del Talco, 68, San Andrés, Madrid",None
3,21739,NaN,NaN,"Calle Pedro Jiménez, San Andrés, Madrid",None
4,21738,40.360559,-3.682858,"Carretera de Villaverde a Vallecas, Los Rosale...","(Carretera de Villaverde a Vallecas, Los Rosal..."
5,21737,NaN,NaN,"geologia, San Andrés, Madrid",None
6,21736,NaN,NaN,"Avenida Real de Pinto, San Andrés, Madrid",None
7,21733,40.350808,-3.688453,"Calle de Martinez Oviol, Los Rosales, Madrid","(Calle de Martínez Oviol, Villaverde Bajo, Los..."
8,21732,NaN,NaN,"De la Plata - Villaverde, San Andrés, Madrid",None
9,21730,40.360404,-3.685349,"Calle de la Unanimidad, 67, Los Rosales, Madrid","(Calle de la Unanimidad, Los Rosales, Villaver..."


In [19]:
import sys
import os
import numpy as np
import pandas as pd
import requests
import time

# Configurar opciones para pandas
pd.set_option("display.max_columns", None)

# Leer el archivo CSV con las direcciones
data = pd.read_csv('../data/houses_Madrid.csv', sep=',', header=0, index_col=0)

# Seleccionar las columnas relevantes para geocodificación
data_map = data.loc[:, ['id', 'latitude', 'longitude', 'raw_address', 'subtitle']]

# Limpiar datos: eliminar filas con direcciones nulas
data_map.dropna(subset=['raw_address'], inplace=True)

# Eliminar duplicados de direcciones
data_map.drop_duplicates(subset=['raw_address'], inplace=True)

# Crear una nueva columna 'address' combinando 'raw_address' y 'subtitle'
data_map['address'] = data_map['raw_address'] + ', ' + data_map['subtitle']
data_map.drop(columns=['raw_address', 'subtitle'], inplace=True)
data_map.reset_index(drop=True, inplace=True)

# Función para hacer la solicitud a la API local
def geocode_local(address):
    api_url = "http://localhost:8080/search"  # URL de la API local
    params = {
        'q': address,  # Dirección a geocodificar
        'format': 'json',  # Formato de respuesta en JSON
        'limit': 1,  # Limitar a 1 resultado
    }

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Lanza una excepción si hay un error HTTP
        data = response.json()
        
        if data:
            # Suponiendo que la respuesta contiene 'lat' y 'lon' para las coordenadas
            latitude = data[0].get('lat')
            longitude = data[0].get('lon')
            return latitude, longitude
        else:
            return None, None  # Si no se encuentra dirección

    except requests.exceptions.RequestException as e:
        print(f"Error al geocodificar la dirección {address}: {e}")
        return None, None  # Si ocurre un error, devolver None

# Aplicar la función de geocodificación a todas las direcciones
data_map['latitude'], data_map['longitude'] = zip(*data_map['address'].apply(geocode_local))

# Verificar cuántas coordenadas son válidas
valid_coordinates = data_map.dropna(subset=['latitude', 'longitude'])
invalid_coordinates = data_map[data_map['latitude'].isnull() | data_map['longitude'].isnull()]

# Imprimir estadísticas de resultados
print(f"Total de coordenadas válidas: {valid_coordinates.shape[0]}")
print(f"Total de coordenadas inválidas: {invalid_coordinates.shape[0]}")

# Guardar los resultados con las coordenadas en un archivo CSV
data_map.to_csv('../data/houses_Madrid_with_coordinates.csv', index=False)
invalid_coordinates.to_csv('../data/houses_Madrid_invalid_coordinates.csv', index=False)

# Mostrar una muestra de los resultados
print(valid_coordinates.head(10))


Total de coordenadas válidas: 6736
Total de coordenadas inválidas: 2930
       id     latitude           longitude  \
0   21742   40.3445124          -3.6894412   
1   21741   40.3538503          -3.6983625   
4   21738   40.3533802          -3.6901152   
7   21733   40.3532352          -3.6899154   
9   21730   40.3613894          -3.6894222   
10  21729  40.35101745  -3.701777508622853   
12  21727   40.3528454          -3.6894826   
13  21719   40.2851537          -3.7934286   
17  21714   40.3533128          -3.6909843   
20  21708   40.3540323          -3.6816541   

                                              address  
0         Calle de Godella, 64, San Cristóbal, Madrid  
1   Calle de la del Manojo de Rosas, Los Ángeles, ...  
4   Carretera de Villaverde a Vallecas, Los Rosale...  
7        Calle de Martinez Oviol, Los Rosales, Madrid  
9     Calle de la Unanimidad, 67, Los Rosales, Madrid  
10           Calle de Anoeta, 63, Los Ángeles, Madrid  
12             Calle San Maca

In [20]:
import pandas as pd

# Cargar el archivo houses_Madrid_with_coordinates.csv
data_with_coords = pd.read_csv('../data/houses_Madrid_with_coordinates.csv')

# Filtrar las filas donde ambas columnas 'latitude' y 'longitude' no son nulas
data_with_coords = data_with_coords.dropna(subset=['latitude', 'longitude'])

# Guardar el archivo limpio
data_with_coords.to_csv('../data/houses_Madrid_cleaned.csv', index=False)

# Imprimir el número de filas antes y después de la limpieza
print(f"Filas originales: {len(data_with_coords) + data_with_coords.isnull().sum().sum()}")
print(f"Filas después de la limpieza: {len(data_with_coords)}")


Filas originales: 6736
Filas después de la limpieza: 6736
